# How to 'discretize' plotly choropleths

In [82]:
import plotly.plotly as py
import plotly.tools as tls

Starting with:

In [83]:
tls.embed('https://plot.ly/~MattSundquist/19045/executions-by-us-state-since-1819/')

In [99]:
MattSundquist19045 = py.get_figure('https://plot.ly/~MattSundquist/19045/executions-by-us-state-since-1819/', raw=True)

### Suggestion 1: with a discrete colorscale

example:

In [85]:
tls.embed('https://plot.ly/~chris/15362/net-energy-imports-as-a-percentage-of-energy-use/')

Discrete colorscales look like:

In [86]:
chris15362 = py.get_figure('https://plot.ly/~chris/15362/net-energy-imports-as-a-percentage-of-energy-use/', raw=True)

chris15362['data'][0]['colorscale']

[[0.0, u'rgb(5,48,97)'],
 [0.18265666689933938, u'rgb(5,48,97)'],
 [0.18265666689933938, u'rgb(33,102,172)'],
 [0.36531333379867875, u'rgb(33,102,172)'],
 [0.36531333379867875, u'rgb(67,147,195)'],
 [0.5479700006980182, u'rgb(67,147,195)'],
 [0.5479700006980182, u'rgb(146,197,222)'],
 [0.7306266675973575, u'rgb(146,197,222)'],
 [0.7306266675973575, u'rgb(209,229,240)'],
 [0.9132833344966969, u'rgb(209,229,240)'],
 [0.9132833344966969, u'rgb(247,247,247)'],
 [0.9277361120805807, u'rgb(247,247,247)'],
 [0.9277361120805807, u'rgb(253,219,199)'],
 [0.9421888896644646, u'rgb(253,219,199)'],
 [0.9421888896644646, u'rgb(244,165,130)'],
 [0.9566416672483484, u'rgb(244,165,130)'],
 [0.9566416672483484, u'rgb(214,96,77)'],
 [0.9710944448322323, u'rgb(214,96,77)'],
 [0.9710944448322323, u'rgb(178,24,43)'],
 [0.9855472224161161, u'rgb(178,24,43)'],
 [0.9855472224161161, u'rgb(103,0,31)'],
 [1.0, u'rgb(103,0,31)']]

Define a function

In [100]:
def make_discrete_colorscale(vals, colors):
    colorscale = []
    
    zmin = vals[0]
    zmax = vals[-1]
    d = float(zmax-zmin)
    
    vals_normed = [(val-zmin)/d for val in vals]
    
    for i, val in enumerate(vals_normed[:-1]):
        colorscale.append([val, colors[i]])
        colorscale.append([val, colors[i+1]])
    
    colorscale.append([vals_normed[-1], colors[-1]])
    
    return colorscale

and make a discrete colorscale:

In [101]:
vals = [0, 10, 100, 1000, 1500]
colors = ['#ffffcc', '#addd8e', '#78c679', '#31a354', '#006837']

colorscale = make_discrete_colorscale(vals, colors)

colorscale

[[0.0, '#ffffcc'],
 [0.0, '#addd8e'],
 [0.006666666666666667, '#addd8e'],
 [0.006666666666666667, '#78c679'],
 [0.06666666666666667, '#78c679'],
 [0.06666666666666667, '#31a354'],
 [0.6666666666666666, '#31a354'],
 [0.6666666666666666, '#006837'],
 [1.0, '#006837']]

update the figure object:

In [102]:
trace = MattSundquist19045['data'][0]

trace['autocolorscale'] = False
trace['colorscale'] = colorscale
trace['zmin'] = 0
trace['zmax'] = 1500
trace['colorbar'] = dict(
    ticks='outside',
    tickmode='array',
    tickvals=vals,
    ticktext=[''] + vals[1::],
    x=0.9,
    title='State executions'
)

# some style suggestions:
# there's a bug at the michigan/ontario boundary that shows up
# with yout set of style attributes

trace['marker']['line']['color'] = 'black'
MattSundquist19045['layout']['geo']['showlakes'] = False

In [103]:
py.iplot(MattSundquist19045, validate=False, filename='discrete-colorscale')

### Suggestion 2: split data into several choropleth traces

In [93]:
zip(vals, colors)

[(0, '#ffffcc'),
 (10, '#addd8e'),
 (100, '#78c679'),
 (1000, '#31a354'),
 (1500, '#006837')]

In [114]:
N = len(vals)
data = []

def make_choropleth(i):
    color = colors[i]
    
    return dict(
        type='choropleth',
        locations=[],
        z=[],
        text=[],
        colorscale=[[0, color], [1, color]],
        colorbar=dict(
            x=0.9,
            y=i/float(N) + 0.2,
            len=1/float(N)
        )
     )

for i in range(N-1):
    val_range = [vals[i], vals[i+1]]
    
    trace_new = make_choropleth(i)
    
    for loc, z, t in zip(trace['locations'], trace['z'], trace['text']):
        if val_range[0] <= z < val_range[1]:
            trace_new['locations'].append(loc)
            trace_new['z'].append(z)
            trace_new['text'].append(text)
        
    data.append(trace_new)
    
fig = dict(
    data=data,
    layout=MattSundquist19045['layout']
)

In [115]:
py.iplot(fig, validate=False, filename='discrete-colorscale2')